<h1 style="text-align: center; font-size: 60px;"> Интеллектуальный анализ и обработка данных</h1>
<hr>
<h2 style="text-align: center;font-size: 40px;"> ЛАБОРАТОРНАЯ РАБОТА №1 <br>
 Разведочный и предиктивный анализ данных на основе регрессионных моделей</h2>
 <hr>

<h3 style="text-align: center; font-size: 30px;">Исполнители</h3>
<p><b>Корзун Николай Вадимович</b> <a href="https://github.com/nikoloskorzun">GitHub</a></p>


<hr>
<h3 style="text-align: center; font-size: 30px;">Постановка задачи</h3>

<p><b>Вариант №12</b></p>
<p>Набор данных различных форм зданий, различающихся по площади остекления, распределению площади остекления, ориентации и другим параметрам. <br>В зависимости от параметров здания других признаков аргументов. Построить регрессионную модель оценки целевого признака требований к тепловой нагрузке зданий:</p>

<ol>
    <li>X1 вещественный, относительная компактность</li>
    <li>X2 вещественный, площадь поверхности</li>
    <li>X3 вещественный, площадь сплошной стены</li>
    <li>X4 вещественный, зона крыши</li>
    <li>X5 вещественный, общая высота</li>
    <li>X6 целочисленный, ориентация</li>
    <li>X7 вещественный, площадь сплошного остекления</li>
    <li>X8 целочисленный, распределение площади остекления</li>    
    <li>Y1 целевой признак, вещественный, отопительная нагрузка</li>
</ol>

a. Пункт 3.h – Нормализация
b. Пункт 4 – гребневая регрессия (регуляризованная линейная регрессия

<h3 style="text-align: center; font-size: 30px;">Выполнение задачи</h3>
<h4 style="text-align: center; font-size: 24px;">Этап 2<br>Загрузка датасета</h4>

In [13]:
names = ['относительная компактность', 'площадь поверхности', 'площадь сплошной стены',
         'зона крыши', 'общая высота', 'ориентация', 'площадь сплошного остекления',
         'распределение площади остекления', 'отопительная нагрузка']


In [14]:
import pandas as pd
import numpy as np


In [15]:
df = pd.read_excel("../V12.xlsx", names = names)
df.head()

,относительная компактность,площадь поверхности,площадь сплошной стены,зона крыши,общая высота,ориентация,площадь сплошного остекления,распределение площади остекления,отопительная нагрузка
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84
